# BANCOS
Este programa simula a chegada de clientes a um banco, que possui alguns postos de atendimento (caixas). A fila é única para todos os caixas.
Os tempos entre chegadas de clientes segue uma distribuição exponencial.
A duração do atendimento a um cliente no caixa segue uma distribuição normal. 

Ao final, você encontra algumas sugestões de melhorias neste programa, que você pode fazer como exercício.

# eventos 
É uma lista de eventos programados, sendo que cada evento
pode ser:


*   -1 é o evento da chegada de um cliente ao banco
*   0, 1, 2 ... é o evento do término do atendimento de um cliente
no caixa 0, 1, 2, ...




# horario_eventos
É a lista dos horários em que ocorrerão os eventos programados na lista **eventos**. 

# caixas
É uma lista dos caixas (postos de atendimentos). Cada caixa pode estar em um dos seguintes estados:
*   "livre"
*   "ocupado"
*   "fechado"





# **fila**
É uma lista que contém os horários de chegada nos clientes que comporão a fila de atendimento. Serve para podermos medir o tempo de espera em fila. 

# **horario**
Variável inteira com o horário atual. O tempo é medido em segundos, contados a partir do instante 0 (instante em que a simulação começou a ser executada).

# **tempos_espera**
Lista que registra o tempo de espera na fila de cada cliente que vem ao banco. Se o cliente não encontrou nenhuma fila, registra zero. Unidade é minutos (e não segundos!). 

In [ ]:
import random # biblioteca para geração de números aleatórios
import time   # biblioteca que possui função para gerar espera no processamento

# "magic funcion" para gráficos aparecerem aqui no Jupyter
%matplotlib inline 


from matplotlib import pyplot as plt  # biblioteca gráfica

random.seed(1) # define semente fixa para geração de números pseudo-aleatórios

#---------------------------------------------------------------------

# Horário corrente (inteiro, em segundos) começa em zero.
horario = 0 

# A lista de ventos começa, de início, com um único evento: a chegada de um
# cliente. Este evento ocorrerá no instante 0.
eventos = [-1]
horario_eventos = [0]

# Aqui está o estado inicial dos postos de atendimento (caixas).
#caixas = ["livre", "livre", "livre"]

# A fila de atendimento começa vazia (ninguém na fila). 
fila = []

# Não há registros de tempos de espera pelos clientes (ninguém chegou ainda!).
tempos_espera = []

#---------------------------------------------------------------------

# Chegada de um novo cliente ao banco
# A função chegada() não recebe nenhum parâmtro e não devolve nenhum resultado.

def chegada(distribuicao, tempo_medio, desvio_padrao):  
    
    # As variáveis abaixo estão no programa principal.
    global horario, eventos, horario_eventos, caixas, fila
    
    # Programa a chegada de um próximo cliente após um tempo aleatório. 
    eventos.append(-1)
    
    if distribuicao == 'N':
        #curva normal -- 59 segundos com desvio padrão de 8 segundos
        h = horario + random.gauss(tempo_medio, desvio_padrao) # normal, especificando média e dp
        horario_eventos.append(h)
        print("Programado evento de chegada de novo cliente aos", round(h,1), "segundos")
    if distribuicao == 'E':
        #curva exponencial -- 59 segundos com desvio padrão de 8 segundos
        h = horario + random.expovariate(1/tempo_medio)
        horario_eventos.append(h)
        print("Programado evento de chegada de novo cliente aos", round(h,1), "segundos")
        

    
    # Se há caixa livre, programa o evento o término do atendimento de cliente
    # recém chegado neste caixa.
    # Se não há caixa livre, cliente vai para fila de espera. 
    if "livre" in caixas:          # verifica aqui se há caixa livre   
        i = caixas.index("livre")    # em havendo caixa livre, descobre qual caixa
        caixas[i] = "ocupado"        # o caixa passa a estar acupado agora
        eventos.append(i)            # gera evento do término do atendimento
        if(distribuicao == 'N'):
            h = horario + random.gauss(tempo_medio, desvio_padrao) # normal, especificando média e dp
        else:
            h = horario + random.expovariate(1/tempo_medio)
        horario_eventos.append(h)    # registra o horário programado para o final
                                   # desse atendimento
        tempos_espera.append(0)      # registra a espera deste cliente (0 seg) 
        print("Programado evento de final de atendimento no caixa ", i, " aos", round(h,1), "segundos")
    else:
      fila.append(horario)         # estando todos os caixas ocupados, 
                                   # põe cliente na fila (anota-se na fila o
                                   # o instante de chegada deste cliente)
        
#---------------------------------------------------------------------     
        
# Trata o eventos do término do atendimento do cliente no caixa i. 
# Se há cliente na fila, aloca-o neste caixa, programando o término do
# atendimento neste caixa. 
# A função saida(i) recebe, como parâmetro o número do caixa terminou de
# atender seis cliente. Mas a função nada retorna como resultado. 

def saida(i):
  
    # As variáveis abaixo estão no programa principal.
    global horario, eventos, horario_eventos, caixas, fila
    
    if len(fila) == 0:        # Caixa foi liberado, mas não há cliente
        caixas[i] = "livre"   # na fila --> muda status deste caixa para livre.
    else:
        # Havendo cliente na fila, registra quantos minutos ele esperou
        # para ser atendido e registra essa informação. Retira o cliente da
        # fila. Programa o final do atendimento do cliente nesse caixa. 
        tempos_espera.append((horario-fila[0])/60) # espera = agora - chegada
        del fila[0]
        eventos.append(i)
        
        if(distribuicao == 'N'):
            y = horario+random.gauss(tempo_medio, desvio_padrao)
        else:
            y = horario+random.expovariate(1/tempo_medio)
        horario_eventos.append(y)
        print("Programado evento de final de atendimento no caixa ", i, " aos", round(y, 1), "segundos")
        
#---------------------------------------------------------------------     

# Descobre qual o próximo evento a tratar na lista de eventos.
# A função proximo_evento() não recebe parâmetros e devolve, como resposta,
# qual é o próximo evento (-1 = chegada de cliente, 0, 1, 2... = término de
# atendimento em um caixa). 
# Para tanto procura em horario-eventos o horário mais próximo (o menor deles). 
# Se há cliente na fila, aloca-o neste caixa,
# programando o término do atendimento neste caixa. 
# A lista de eventos nunca estará vazia, por construção. 
# Essa função retorna o tipo do próximo evento (-1, 0, 1, 2..). 

def proximo_evento():
  
    global horario, eventos, horario_eventos, caixas, fila
    
    
    pos = horario_eventos.index(min(horario_eventos)) # index do próximo evento
    evento = eventos[pos]                             # obtém evento
    horario = horario_eventos[pos]                    # obtém horário do evento
    if evento == -1:
      e = "chegada de cliente"
    else:
      e = "término de atendimento no caixa " + str(evento)
    
    print("Tratando evento = " + e + ", horário do evento =", round(horario, 1), "segundos")
    eventos.pop(pos)            # apaga o referido evento da lista de eventos
    horario_eventos.pop(pos)    # apaga horário do evento na lista dos horários
    return evento               # devolve o evento para o programa principal


#Verifica se um cliente é senior num intervalo de 0% à 100% para atribuir-lhe prioridade no atendimento
#Esta prioridade é válida para o Caixa 0
def verifica_cliente_senior(probabilidade_senior):

    lista = list()
    count = 1
    for count in range(100):
        if probabilidade_senior > 0:
            lista.append(True)
            probabilidade_senior = probabilidade_senior - 1
        else:
            lista.append(False)
        count = count + 1

    return random.choice(lista)


# -----------------------------------------------------------------------
    
# PROGRAMA PRINCIPAL

# Permite ao usuário selecionar a distribuição de probabilidade para os tempos entre chegadas de clientes
while True:
    print('Qual a distribuição de probabilidade que gostaria de considerar para este exercício? Considere "N" para distribuição NORMAL e "E" para EXPONENCIAL')
    distribuicao = input()
    if distribuicao == 'N':
        while True:
            print('Qual o tempo medio (em segundos)?')
            tempo_medio = input()
            if tempo_medio.isdigit():
                tempo_medio = int(tempo_medio)
                break
        
        while True:
            print('Qual o desvio padrao (em segundos)?')
            desvio_padrao = input()
            if desvio_padrao.isdigit():
                desvio_padrao = int(desvio_padrao)
                break
        break
    
    if distribuicao == 'E':
        while True:
            print('Qual o tempo medio (em segundos)?')
            tempo_medio = input()
            if tempo_medio.isdigit():
                tempo_medio = int(tempo_medio)
                break
            
        desvio_padrao = None
        break


while True:
    print('Informe o tamanho máximo da fila de clientes no banco:')
    tam_max_fila = input()
    if tam_max_fila.isdigit():
        tam_max_fila = int(tam_max_fila)
        break


while True:
    print('Informe a quantidade de caixas disponíveis:')
    caixas_disponiveis = input()
    if caixas_disponiveis.isdigit():
        caixas = list()
        caixas_index = 0
        for caixas_index in range(int(caixas_disponiveis)):
            caixas.append("livre")
        caixas_disponiveis = int(caixas_disponiveis)
        break

while True:
    print('Informe a probabilidade (de 0 a 100) de um cliente ser senior:')
    probabilidade_senior = input()
    if probabilidade_senior.isdigit():
        probabilidade_senior = int(probabilidade_senior)
        break


# Trata eventos programados durante as 3 primeiras horas.
while horario < 60*60*3:  # 3 horas
 
  # time.sleep(1) # para melhorar a apresentação (ser quiser fazê-la pausada)
  
  x = proximo_evento()   # descobre qual é e trata o próximo evento
  if x == -1:
    if distribuicao == 'N':
        chegada(distribuicao, tempo_medio, desvio_padrao)
    if distribuicao == 'E':
        chegada(distribuicao, tempo_medio, None)
  else:  # é Saída
     saida(x)
      
  print("Tamanho da fila = ", len(fila)) # imprime tamanho da fila
  print("Caixas: " , caixas)             # imprime situação dos caixas
  
print()
print()
print("**** Tempo máximo de espera na fila (minutos): ", round(max(tempos_espera), 1))
print("**** Tempo médio de espera na fila (minutos): ", round(sum(tempos_espera)/len(tempos_espera), 1))
print()
print()

# gera histograma dos tempos de espera na fila
plt.hist(tempos_espera, bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])
plt.title('Histograma dos tempos de espera na fila')
plt.xlabel("Tempo de espera na fila para ser atendido (minutos)")
plt.ylabel("Contagem de clientes")
plt.grid(True)
plt.show()

# gera line plot dos tempos de espera na fila
plt.plot(tempos_espera, color = "r")
plt.title('Timeplot dos tempos de espera na fila')
plt.xlabel("Número de ordem de chegada dos clientes")
plt.ylabel("Tempo de espera na fila para ser atendido (minutos)")
plt.grid(True)
plt.grid(True)
plt.show()

##### 

# **Exercícios**

Evolua a programa acima.

a) Permita que o usuário escolha a distribuições de probabilidade para os tempos entre chegadas de clientes. No programa acima, usamos a Distribuição Normal (parâmetros são o tempo médio e o desvio-padrão). Mas, uma melhor distribuição de probabilidades para esses tempos é a Distribuição Exponencial, cujo único parâmetro é a média (o desvio-padrão é sempre igual à média nesta distribuição). Permita também que o usuário escolha os parâmetros da distribuição.

b) Permita que o usuário escolha os parâmetros da distribuição normal de probabilidades para a duração do atendimento no caixa. 

c) Faça com que o programa informe o tamanho máximo da fila de clientes no banco.

d) Permita que o usuário defina o número de caixas disponíveis. 

e) Introduza o conceito de cliente senior. Usuário deve informar a probabilidade de um cliente ser senior. Faça com que o caixa 0 dê prioridade aos clientes senior.

Dicas:

*   normal: random.gauss(média, dp)
*   uniforme discreta: random.randint(mínimo, máximo)
*   uniforme contínua: random.uniform(mínimo, máximo)
*   exponencial: random.expovariate(1/média)








In [ ]:
import random # biblioteca para geração de números aleatórios
import time   # biblioteca que possui função para gerar espera no processamento

# "magic funcion" para gráficos aparecerem aqui no Jupyter
%matplotlib inline 


from matplotlib import pyplot as plt  # biblioteca gráfica

random.seed(1) # define semente fixa para geração de números pseudo-aleatórios

#---------------------------------------------------------------------

# Horário corrente (inteiro, em segundos) começa em zero.
horario = 0 

# A lista de ventos começa, de início, com um único evento: a chegada de um
# cliente. Este evento ocorrerá no instante 0.
eventos = [-1]
horario_eventos = [0]

# Aqui está o estado inicial dos postos de atendimento (caixas).
caixas = ["livre", "livre", "livre"]

# A fila de atendimento começa vazia (ninguém na fila). 
fila = []

# Não há registros de tempos de espera pelos clientes (ninguém chegou ainda!).
tempos_espera = []


In [ ]:
def chegada():  
    
    # As variáveis abaixo estão no programa principal.
    global horario, eventos, horario_eventos, caixas, fila
    
    # Programa a chegada de um próximo cliente após um tempo aleatório. 
    eventos.append(-1)
    h = horario + random.gauss(59,8) # normal, especificando média e dp
    horario_eventos.append(h)
    print("Programado evento de chegada de novo cliente aos", round(h,1), "segundos")
    
    # Se há caixa livre, programa o evento o término do atendimento de cliente
    # recém chegado neste caixa.
    # Se não há caixa livre, cliente vai para fila de espera. 
    if "livre" in caixas:          # verifica aqui se há caixa livre   
      i = caixas.index("livre")    # em havendo caixa livre, descobre qual caixa
      caixas[i] = "ocupado"        # o caixa passa a estar acupado agora
      eventos.append(i)            # gera evento do término do atendimento
      h = horario + random.gauss(180, 30) # normal, especificando média e dp
      horario_eventos.append(h)    # registra o horário programado para o final
                                   # desse atendimento
      tempos_espera.append(0)      # registra a espera deste cliente (0 seg) 
      print("Programado evento de final de atendimento no caixa ", i, " aos", round(h,1), "segundos")
    else:
      fila.append(horario)         # estando todos os caixas ocupados, 
                                   # põe cliente na fila (anota-se na fila o
                                   # o instante de chegada deste cliente)
        
#---------------------------------------------------------------------     
        
# Trata o eventos do término do atendimento do cliente no caixa i. 
# Se há cliente na fila, aloca-o neste caixa, programando o término do
# atendimento neste caixa. 
# A função saida(i) recebe, como parâmetro o número do caixa terminou de
# atender seis cliente. Mas a função nada retorna como resultado. 

def saida(i):
  
    # As variáveis abaixo estão no programa principal.
    global horario, eventos, horario_eventos, caixas, fila
    
    if len(fila) == 0:        # Caixa foi liberado, mas não há cliente
        caixas[i] = "livre"   # na fila --> muda status deste caixa para livre.
    else:
        # Havendo cliente na fila, registra quantos minutos ele esperou
        # para ser atendido e registra essa informação. Retira o cliente da
        # fila. Programa o final do atendimento do cliente nesse caixa. 
        tempos_espera.append((horario-fila[0])/60) # espera = agora - chegada
        del fila[0]
        eventos.append(i)
        y = horario+random.gauss(180, 30)
        horario_eventos.append(y)
        print("Programado evento de final de atendimento no caixa ", i, " aos", round(y, 1), "segundos")
        
#---------------------------------------------------------------------     

# Descobre qual o próximo evento a tratar na lista de eventos.
# A função proximo_evento() não recebe parâmetros e devolve, como resposta,
# qual é o próximo evento (-1 = chegada de cliente, 0, 1, 2... = término de
# atendimento em um caixa). 
# Para tanto procura em horario-eventos o horário mais próximo (o menor deles). 
# Se há cliente na fila, aloca-o neste caixa,
# programando o término do atendimento neste caixa. 
# A lista de eventos nunca estará vazia, por construção. 
# Essa função retorna o tipo do próximo evento (-1, 0, 1, 2..). 

def proximo_evento():
  
    global horario, eventos, horario_eventos, caixas, fila
    
    
    pos = horario_eventos.index(min(horario_eventos)) # index do próximo evento
    evento = eventos[pos]                             # obtém evento
    horario = horario_eventos[pos]                    # obtém horário do evento
    if evento == -1:
      e = "chegada de cliente"
    else:
      e = "término de atendimento no caixa " + str(evento)
    
    print("Tratando evento = " + e + ", horário do evento =", round(horario, 1), "segundos")
    eventos.pop(pos)            # apaga o referido evento da lista de eventos
    horario_eventos.pop(pos)    # apaga horário do evento na lista dos horários
    return evento               # devolve o evento para o programa principal
    

In [ ]:
print("horario: " + str(horario))
print()
print("eventos: " + str(eventos))
print()
print("horario_eventos: " + str(horario_eventos))
print()
print("caixas: " + str(caixas))
print()
print("fila: " + str(fila))
print()
print("tempos_espera: " + str(tempos_espera))

In [ ]:
horario_eventos.index(min(horario_eventos))

In [ ]:
proximo_evento()

In [ ]:
print("horario: " + str(horario))
print()
print("eventos: " + str(eventos))
print()
print("horario_eventos: " + str(horario_eventos))
print()
print("caixas: " + str(caixas))
print()
print("fila: " + str(fila))
print()
print("tempos_espera: " + str(tempos_espera))

In [ ]:
chegada()

In [ ]:
print("horario: " + str(horario))
print()
print("eventos: " + str(eventos))
print()
print("horario_eventos: " + str(horario_eventos))
print()
print("caixas: " + str(caixas))
print()
print("fila: " + str(fila))
print()
print("tempos_espera: " + str(tempos_espera))

In [ ]:
pos = horario_eventos.index(min(horario_eventos))
evento = eventos[pos]                             # obtém evento
horario = horario_eventos[pos]
print(evento)
print(horario)

In [ ]:
proximo_evento()

In [ ]:
print("horario: " + str(horario))
print()
print("eventos: " + str(eventos))
print()
print("horario_eventos: " + str(horario_eventos))
print()
print("caixas: " + str(caixas))
print()
print("fila: " + str(fila))
print()
print("tempos_espera: " + str(tempos_espera))

In [ ]:
chegada()

In [ ]:
print("horario: " + str(horario))
print()
print("eventos: " + str(eventos))
print()
print("horario_eventos: " + str(horario_eventos))
print()
print("caixas: " + str(caixas))
print()
print("fila: " + str(fila))
print()
print("tempos_espera: " + str(tempos_espera))

In [ ]:
pos = horario_eventos.index(min(horario_eventos))
evento = eventos[pos]                             # obtém evento
horario = horario_eventos[pos]
print(pos)
print(evento)
print(horario)

In [ ]:
proximo_evento()

In [ ]:
print("horario: " + str(horario))
print()
print("eventos: " + str(eventos))
print()
print("horario_eventos: " + str(horario_eventos))
print()
print("caixas: " + str(caixas))
print()
print("fila: " + str(fila))
print()
print("tempos_espera: " + str(tempos_espera))

In [ ]:
chegada()

In [ ]:
print("horario: " + str(horario))
print()
print("eventos: " + str(eventos))
print()
print("horario_eventos: " + str(horario_eventos))
print()
print("caixas: " + str(caixas))
print()
print("fila: " + str(fila))
print()
print("tempos_espera: " + str(tempos_espera))

In [ ]:
proximo_evento()

In [ ]:
print("horario: " + str(horario))
print()
print("eventos: " + str(eventos))
print()
print("horario_eventos: " + str(horario_eventos))
print()
print("caixas: " + str(caixas))
print()
print("fila: " + str(fila))
print()
print("tempos_espera: " + str(tempos_espera))

In [ ]:
chegada()

In [ ]:
print("horario: " + str(horario))
print()
print("eventos: " + str(eventos))
print()
print("horario_eventos: " + str(horario_eventos))
print()
print("caixas: " + str(caixas))
print()
print("fila: " + str(fila))
print()
print("tempos_espera: " + str(tempos_espera))

In [ ]:
proximo_evento()

In [ ]:
print("horario: " + str(horario))
print()
print("eventos: " + str(eventos))
print()
print("horario_eventos: " + str(horario_eventos))
print()
print("caixas: " + str(caixas))
print()
print("fila: " + str(fila))
print()
print("tempos_espera: " + str(tempos_espera))

In [ ]:
saida(0)

In [ ]:
print("horario: " + str(horario))
print()
print("eventos: " + str(eventos))
print()
print("horario_eventos: " + str(horario_eventos))
print()
print("caixas: " + str(caixas))
print()
print("fila: " + str(fila))
print()
print("tempos_espera: " + str(tempos_espera))